# PDB Preparation

In [ ]:
import MDAnalysis as mda
from MDAnalysis.analysis import align
import numpy as np
import nglview as nv
import spc_os 
import os
import gmxapi as gmx

In [ ]:
roseta_dir = '../models/raw_pdb/rosetta_5VKE/'
data_dir = '../data/raw/'

# Starting PDB

Took initial structure obtained from a previous project of 5VKE with and without lipids. It is a starting structure which was prepared and before it is passed through charmm-gui and had additional changes.
This structure is `../models/raw_pdb/full_before_rosetta.pdb`

# ROSETTA
Add residues so the sequence is between 25 and 121 to the 5VKE pdb with rosetta `models/raw_pdb/rosetta` contains the files.
The starting structure is `../models/raw_pdb/full_before_rosetta.pdb` without the caps and just with `segid PROA`

## Renumber

In [ ]:
start = mda.Universe(f'{roseta_dir}/output_files/input_0001.pdb')

In [ ]:
start.residues.resids = np.arange(25, start.residues.n_residues+25)

## Tetramerize

In [ ]:
%ls ../models/raw_pdb/

In [ ]:
ref = mda.Universe('../models/raw_pdb/full_before_rosetta_5VKE.pdb')

In [ ]:
merge1 = start.copy()
align.alignto(start,ref,select="name CA and (segid PROB or segid A) and resid 62:73")
merge1.segments.segids = 'B'
merge2 = start.copy()
align.alignto(start,ref,select="name CA and (segid PROC or segid A) and resid 62:73")
merge2.segments.segids = 'C'
merge3 = start.copy()
align.alignto(start,ref,select="name CA and (segid PROD or segid A) and resid 62:73")
merge3.segments.segids = 'D'
start.segments.segids = 'A'
merged = mda.Merge(start.select_atoms('all'), merge1.select_atoms('all'), merge2.select_atoms('all'), merge3.select_atoms('all'), ref.select_atoms('not protein'))

In [ ]:
merged.atoms.write("../models/raw_pdb/rosetta_merged_renumbered_5VKE_lb.pdb")
merged = mda.Merge(start.select_atoms('all'), merge1.select_atoms('all'), merge2.select_atoms('all'), merge3.select_atoms('all'), ref.select_atoms('not protein and not resname DOPG'))
merged.atoms.write("../models/raw_pdb/rosetta_merged_renumbered_5VKE.pdb")

In [ ]:
view = nv.show_mdanalysis(merged)
view.add_licorice("TYR")
view

## Upload CHARMM-GUI
+ Caps ACE, CT3
+ Protonate E71, E118, E120, H25
+ ppm server
+ generate pore water
+ 80Å 3:1 PE/PG chains DO
+ 0.1 NaCl
+ CHARMM36, gromacs
+ lb: 1881606330 not_lb:1881601122 

# Modifications
+ add to step5_input.pdb the pbc
+ add Na such that half is K.
+ removed waters that are in the lipids.
+ add the helix restrains

In [ ]:
%%bash

for sys in 5VKE_25-122 5VKE_lb_25-122
do 
mkdir -p ../data/raw/$sys/opt
mkdir -p ../data/raw/$sys/eq
for file in charmm_final.pdb topol.top toppar
do
ln -s ../../../models/charmm-gui/$sys/$file ../data/raw/$sys/$file
done
done

In [ ]:
%%bash

for sys in 5VKE_25-122 5VKE_lb_25-122
do 
ln -s ../../../../models/mdp/opt.mdp ../data/raw/$sys/opt/opt.mdp
for i in $(seq 0 7)
do
ln -s ../../../../models/mdp/NPTres$i.mdp ../data/raw/$sys/eq/NPTres$i.mdp
done
done

In [ ]:
systems = ["5VKE_25-122", "5VKE_lb_25-122"]

In [ ]:
selections = {
    "all": "all",
    "protein": "protein",
    "waters": "name OH2 H1 H2 and resname TIP3",
    "waters_or_ions": "(name OH2 H1 H2 and resname TIP3) or resname SOD POT CLA CA",
    "lipids": "resname DOPG DOPE POPC POPS CHL1 POPG",
    "PROA": "segid PROA",
    "PROB": "segid PROB",
    "PROC": "segid PROC",
    "PROD": "segid PROD",
}

In [ ]:
for sys in systems:
    ndx_path = f"{data_dir}{sys}/index.ndx"
    spc_os.remove(ndx_path)
    univ = mda.Universe(f"{data_dir}{sys}/charmm_final.pdb")
    for selection in selections.keys():
        univ.select_atoms(selections[selection]).write(
            ndx_path, name=selection, mode="a"
        )

## Optimize

In [ ]:
for sys in systems:
    path = f"{data_dir}{sys}/opt"
    cdminus = os.getcwd()
    os.chdir(path)
    grompp = gmx.commandline_operation(
        "gmx",
        "grompp",
        input_files={
            "-f": "opt.mdp",
            "-p": "../topol.top",
            "-c": "../charmm_final.pdb",
            "-r": "../charmm_final.pdb",
            "-n": "../index.ndx",
            "-pp": "topol.top",
        },
        output_files={"-o": "topol.tpr"},
    )
    grompp.run()
    if grompp.output.erroroutput.result() != "":
        print(grompp.output.erroroutput.result())
    os.chdir(cdminus)

In [ ]:
for sys in systems:
    path = f"{data_dir}{sys}/opt"
    cdminus = os.getcwd()
    os.chdir(path)
    simulation_input = gmx.read_tpr("topol.tpr")
    md = gmx.commandline_operation("gmx", "mdrun", input_files={"-s": "topol.tpr"})
    md.run()
    os.chdir(cdminus)